In [25]:

import albumentations
import torch
import torch.nn.functional as F
import typing
from torch.utils.data import DataLoader
from torch.optim import Adam
from models.unet import UNet
from src.dataset import MidvDataset
from pathlib import Path 
from models.loss import dice_loss

In [4]:
device = 'cuda'

In [5]:
model = UNet(n_class = 1)
model = model.cuda()

In [6]:
list(model.children())

[Sequential(
   (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): ReLU(inplace=True)
   (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (3): ReLU(inplace=True)
 ), Sequential(
   (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): ReLU(inplace=True)
   (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (3): ReLU(inplace=True)
 ), Sequential(
   (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): ReLU(inplace=True)
   (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (3): ReLU(inplace=True)
 ), Sequential(
   (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (1): ReLU(inplace=True)
   (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
   (3): ReLU(inplace=True)
 ), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Upsample(scale_factor=2.0, mode=bilinear)

In [7]:
list_images = list(Path('data_processed/images').rglob('*.jpg'))
list_masks = list(Path('data_processed/labels').rglob('*.png'))

In [8]:
list_masks = [str(el) for el in list_masks]
list_images = [str(el) for el in list_images]


In [9]:
samples = list(zip(list_masks, list_images))
samples = [tuple(el) for el in samples]

In [10]:
optimizer = Adam(model.parameters(), lr = 0.001)

In [11]:
device = 'cuda'

In [23]:
def train_model(model , optimizer, scheduler , num_epochs, samples) -> None:
    dataset = MidvDataset(samples = samples, transform = albumentations.Compose( [albumentations.LongestMaxSize(max_size=128, p=1)], p=1  ))
    dataloader = DataLoader(dataset,  batch_size = 1, shuffle = True, num_workers = 0)
    model = model.cuda()
    for epoch in range(num_epochs):
        #since = time.time()
        model.train()

        #metrics = defaultdict(float)
        for res in dataloader:
            inputs = res['features'].float().cuda()
            print(inputs.shape)
            labels = res['masks'].float().cuda()
            print(labels.shape)
            optimizer.zero_grad()

            output = model(inputs)
            loss = dice_loss(output, labels)
            loss.backward()
            optimizer.step()


    #model.load_state_dict(best_model_wts)
    return model 


In [24]:
train_model(model = model, optimizer = optimizer, scheduler = None, num_epochs = 10, samples = samples)

torch.Size([1, 3, 128, 72])
torch.Size([1, 1, 128, 72])


NameError: name 'dice_loss' is not defined

In [ ]:
from torchsummary import summary

In [ ]:
import torch

In [20]:
torch.cuda.empty_cache()

In [ ]:

#model = UNet(n_class = 1)
#model = model.cuda()
summary(model, input_size = (3, 32, 32))